# [실습] RAG 파이프라인 성능 평가하기   

지금까지 RAG의 성능을 높이기 위한 다양한 방법에 대해 알아봤는데요.   
실제 RAG의 성능은 어떻게 측정해야 할까요?

이번 실습에서는 정답이 존재하는 RAG 데이터를 이용해, 성능을 평가하는 과정에 대해 알아보겠습니다.

### 라이브러리 설치  

랭체인 관련 라이브러리와 벡터 데이터베이스 라이브러리를 설치합니다.   

In [ ]:
!pip install sacrebleu ragas dotenv jsonlines langchain langchain-openai langchain-community beautifulsoup4 langchain_chroma chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.6/317.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.8 MB/s eta 0:00:00
   ━

세션 재시작이 필요합니다.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('env', override=True)

if os.environ.get('OPENAI_API_KEY'):
    print('OpenAI API 키 확인')

OpenAI API 키 확인


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature = 0.7, max_tokens = 4096)

RAG의 평가를 위해서는 정답이 있는 Q/A 데이터가 필요합니다.   
실습 시트에서 eval.jsonl을 다운로드하여 불러옵니다.

In [ ]:
# jsonl 파일 불러오기
import jsonlines
from langchain.schema import Document

def load_docs_from_jsonl(file_path):
    documents = []
    with jsonlines.open(file_path, mode="r") as reader:
        for doc in reader:
            documents.append(Document(**doc))
    return documents

preprocessed_docs = load_docs_from_jsonl("eval.jsonl")

Chunking을 수행합니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)

chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

643


Embedding 모델을 구성합니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings
openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

ChromaDB를 구성합니다.

In [ ]:
from langchain_chroma import Chroma

Chroma().delete_collection() # (메모리에 저장하는 경우) 기존 데이터 삭제

# DB 구성하기
db = Chroma(embedding_function=openai_embeddings,
            persist_directory="./chroma_OpenAI",
            collection_metadata={'hnsw:space':'l2'},
            )

DB에 document를 추가합니다.

In [ ]:
print(len(chunks))
# 300,000 토큰 제한

# 100개씩 추가
for i in range(0, len(chunks), 100):
    db.add_documents(chunks[i:min(i+100, len(chunks))])

643


db로부터 retriever를 구성합니다.

In [ ]:
retriever = db.as_retriever(search_kwargs={'k':5})
# Chunk Size * K = Context 글자

RAG를 위한 간단한 프롬프트를 작성합니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])

prompt.pretty_print()

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
정확한 답변을 제공하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


RAG를 수행하기 위한 Chain을 만듭니다.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def format_docs(docs):
    return " \n---\n ".join(['URL: '+ doc.metadata['source'] + '\nContent: '+ doc.page_content+ '\n' for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# RAGAS 사용하기


RAGAS 는 다양한 메트릭을 통한 RAG의 성능 평가를 지원합니다.


각각의 메트릭은 아래의 링크에서 확인할 수 있습니다.

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

시트에서 평가 데이터셋을 불러옵니다.   
Question/Ground Truth의 구성입니다.

In [ ]:
# Best: Retrieval의 Ground Truth, Generation의 Ground Truth가 모두 존재

import pandas as pd
df = pd.read_csv('./eval.csv')
eval_dataset = df.to_dict('list')
eval_dataset

{'questions': ['도메인 특화 언어 모델이란 무엇입니까? 어떤 예시가 있나요?',
  '인공지능의 최근 발전 방식은? 관련 링크도 보여주세요',
  '제미나이 2.5 버전의 장점은 무엇입니까?',
  '고니 모델의 파라미터 수는 몇 개입니까?',
  '업스테이지의 대표는 누구입니까?',
  'AI 가전의 3가지 핵심가치는?',
  '지브리 풍의 그림을 그려주는 AI는 무엇입니까?',
  'ChatGPT가 처한 저작권 위기는 무엇입니까?'],
 'ground_truths': ['도메인 특화 언어 모델은 특정 산업·전문 분야의 지식, 데이터, 용어를 집중적으로 학습해 그 분야 업무에 최적화된 AI 언어 모델입니다. 범용 LLM보다 해당 분야에서 더 전문적인 답변을 내고, 학습 비용·시간을 줄일 수 있는 것이 특징입니다.\n\n예시\n- 의료 특화 모델: 환자 의료데이터와 임상시험 결과로 학습해 암 진단·치료 보조\n- 법률 특화 모델: 판례·계약서·법규를 학습해 법률가의 연구 시간 단축\n- S2W 다크버트(DarkBERT): 다크웹 특화 모델(세계 최초). 온톨로지+RAG 결합해 정확도 향상, 기술을 제조(현대제철)·유통(롯데멤버스) 등으로 확장\n- MathGPT(업스테이지·콴다·KT): 수학 도메인 특화 LLM. MATH, GSM8K 벤치마크에서 동급 모델 및 GPT-4/ChatGPT 대비 우수 성능을 주장\n- 딥서치 ‘챗딥서치’: 기업 도메인 특화 대화형 LLM. 재무·실적·뉴스·공시·특허 등 약 20억건 기업 데이터 기반으로 Q&A 제공, 검색 결합으로 환각 최소화\n- KISTI ‘고니’: 한국 과학기술 특화 생성형 LLM. RAG 기능으로 환각 최소화\n- 포티투마루 도메인 특화 설치형 LLM/DocuAgent42: 산업별 요구에 맞춘 설치형 특화 LLM과 생성형 AI 솔루션',
  '최근 AI는 추론을 먼저 수행하는 ‘사고형(thinking) 아키텍처’, 멀티모달과 초장문맥 통합, 특정 분야에 최적화된 도메인 특화(sLLM)+RAG 결합

구성된 RAG 체인을 이용해, RAGAS의 Evaluate 형태로 변환합니다.

In [ ]:
questions, ground_truths = eval_dataset['questions'], eval_dataset['ground_truths']

In [ ]:
for i in range(len(questions)):
    print(f'#{i}')
    print(f'Question: {questions[i]}\n')
    print(f'Ground Truth: {ground_truths[i]}\n')
    print('-----------')

#0
Question: 도메인 특화 언어 모델이란 무엇입니까? 어떤 예시가 있나요?

Ground Truth: 도메인 특화 언어 모델은 특정 산업·전문 분야의 지식, 데이터, 용어를 집중적으로 학습해 그 분야 업무에 최적화된 AI 언어 모델입니다. 범용 LLM보다 해당 분야에서 더 전문적인 답변을 내고, 학습 비용·시간을 줄일 수 있는 것이 특징입니다.

예시
- 의료 특화 모델: 환자 의료데이터와 임상시험 결과로 학습해 암 진단·치료 보조
- 법률 특화 모델: 판례·계약서·법규를 학습해 법률가의 연구 시간 단축
- S2W 다크버트(DarkBERT): 다크웹 특화 모델(세계 최초). 온톨로지+RAG 결합해 정확도 향상, 기술을 제조(현대제철)·유통(롯데멤버스) 등으로 확장
- MathGPT(업스테이지·콴다·KT): 수학 도메인 특화 LLM. MATH, GSM8K 벤치마크에서 동급 모델 및 GPT-4/ChatGPT 대비 우수 성능을 주장
- 딥서치 ‘챗딥서치’: 기업 도메인 특화 대화형 LLM. 재무·실적·뉴스·공시·특허 등 약 20억건 기업 데이터 기반으로 Q&A 제공, 검색 결합으로 환각 최소화
- KISTI ‘고니’: 한국 과학기술 특화 생성형 LLM. RAG 기능으로 환각 최소화
- 포티투마루 도메인 특화 설치형 LLM/DocuAgent42: 산업별 요구에 맞춘 설치형 특화 LLM과 생성형 AI 솔루션

-----------
#1
Question: 인공지능의 최근 발전 방식은? 관련 링크도 보여주세요

Ground Truth: 최근 AI는 추론을 먼저 수행하는 ‘사고형(thinking) 아키텍처’, 멀티모달과 초장문맥 통합, 특정 분야에 최적화된 도메인 특화(sLLM)+RAG 결합, 그리고 연구·업무 자동화를 지향하는 방향으로 발전하고 있습니다.

주요 흐름
- 사고형(추론 중심) 모델: 응답 전 내부 사고 과정을 거쳐 논리적 결론을 내리도록 설계해 정확도와 안정성을 높임. 구글 제미나이 2.5가 대표적 사례로, 여러 벤치마크에서 경쟁 모델을 상회하고 추

RAG 체인을 실행해, 테스트 문제에 대한 답변을 생성합니다.

In [ ]:
from tqdm import tqdm

dataset = []

for query,reference in tqdm(zip(questions,ground_truths)):

    relevant_docs = [doc.page_content for doc in retriever.invoke(query)]
    response = rag_chain.invoke(query)
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )
    # 질문 / 검색 결과 / RAG의 답변 / 정답

8it [00:29,  3.73s/it]


In [ ]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)
evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=8)

RAGAS는 LLM을 이용해 정답과 답변을 개별 Claim(주장)으로 분할합니다.

이후, `LLMContextRecall`, `Faitufulness`, `FactualCorrectness` 등의 다양한 메트릭을 통해 RAG 파이프라인의 성능을 평가합니다.   
LLM 기반의 방법이므로 평가 LLM의 선정이 중요하며, 절대 수치보다는 상대적 비교가 효과적입니다.


- Context Recall: 정답의 Claim들이 모두 검색됐는가
- Faithfulness : 답변의 Claim이 얼마나 검색 결과에 근거했는가
- Factual Correctness : 정답과 답변의 Claim이 얼마나 일치하는가
- Bleu Score : 정답과 답변 키워드가 얼마나 일치하는가
- Semantic Sim : 정답 답변 임베딩이 얼마나 가까운가   


In [ ]:
from ragas import evaluate
from ragas.llms.base import llm_factory
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.embeddings import OpenAIEmbeddings as ragas_OpenAIEmbeddings

from ragas.metrics import BleuScore, SemanticSimilarity
from openai import AsyncOpenAI

client = AsyncOpenAI()

# https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

# 평가자 LLM
evaluator_llm =  llm_factory('gpt-4.1')

# 평가자 Embedding
evaluator_embeddings = ragas_OpenAIEmbeddings(model = 'text-embedding-3-large', client = client)
semantic_scorer = SemanticSimilarity(embeddings = evaluator_embeddings)


result = evaluate(dataset=evaluation_dataset,
                  metrics=[BleuScore(), LLMContextRecall(), semantic_scorer, Faithfulness(), FactualCorrectness()],
                  llm=evaluator_llm,
                  embeddings = evaluator_embeddings
                  )
result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'bleu_score': 0.2173, 'context_recall': 0.9031, 'semantic_similarity': 0.8369, 'faithfulness': 0.9817, 'factual_correctness(mode=f1)': 0.7975}

In [ ]:
result.scores

[{'bleu_score': 0.32282138800401855,
  'context_recall': 0.5555555555555556,
  'semantic_similarity': 0.8693153439421648,
  'faithfulness': 1.0,
  'factual_correctness(mode=f1)': np.float64(0.54)},
 {'bleu_score': 0.02132081946882163,
  'context_recall': 0.7692307692307693,
  'semantic_similarity': 0.8012324071208237,
  'faithfulness': 0.9090909090909091,
  'factual_correctness(mode=f1)': np.float64(0.56)},
 {'bleu_score': 2.0680196149407996e-11,
  'context_recall': 0.9,
  'semantic_similarity': 0.8308452242306487,
  'faithfulness': 0.9444444444444444,
  'factual_correctness(mode=f1)': np.float64(0.69)},
 {'bleu_score': 0.05604233375480574,
  'context_recall': 1.0,
  'semantic_similarity': 0.8303648247053748,
  'faithfulness': 1.0,
  'factual_correctness(mode=f1)': np.float64(1.0)},
 {'bleu_score': 1.0000000000000004,
  'context_recall': 1.0,
  'semantic_similarity': 0.9999981718079983,
  'faithfulness': 1.0,
  'factual_correctness(mode=f1)': np.float64(1.0)},
 {'bleu_score': 0.2460137

In [ ]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,bleu_score,context_recall,semantic_similarity,faithfulness,factual_correctness(mode=f1)
0,도메인 특화 언어 모델이란 무엇입니까? 어떤 예시가 있나요?,[오히려 공개된 모델을 활용해 특정 서비스를 개발하는 것이 가능성이 높다는 설명이다...,"도메인 특화 언어 모델이란 특정 산업이나 전문 분야의 지식, 데이터, 용어를 AI에...","도메인 특화 언어 모델은 특정 산업·전문 분야의 지식, 데이터, 용어를 집중적으로 ...",3.228214e-01,0.555556,0.869315,1.000000,0.54
1,인공지능의 최근 발전 방식은? 관련 링크도 보여주세요,"[[AI시대, 가치 네트워크에 주목하라] 한국 과학기술 특화 `고니` 공개… 확장되...",인공지능의 최근 발전 방식은 다음과 같습니다.\n\n1. **도메인 특화 대규모언어...,"최근 AI는 추론을 먼저 수행하는 ‘사고형(thinking) 아키텍처’, 멀티모달과...",2.132082e-02,0.769231,0.801232,0.909091,0.56
2,제미나이 2.5 버전의 장점은 무엇입니까?,[구글 제미나이 2.5프로 공개.. “가장 지능적인 모델” 입력2025.03.26....,제미나이 2.5 버전의 장점은 다음과 같습니다.\n\n1. **가장 지능적인 모델*...,다음이 제미나이 2.5의 주요 장점입니다.\n\n- 고도화된 추론(사고) 능력: 응...,2.068020e-11,0.900000,0.830845,0.944444,0.69
3,고니 모델의 파라미터 수는 몇 개입니까?,[AI models with fewer parameters and graphic p...,고니 모델의 파라미터 수는 80억 개(8B)입니다.,고니 모델은 8B(매개변수 80억개) 모델 크기입니다.,5.604233e-02,1.000000,0.830365,1.000000,1.00
4,업스테이지의 대표는 누구입니까?,[언어 모델 '솔라(SOLAR)'로 세계 1위의 거대언어모델(LLM)을 만든 업스테...,업스테이지의 대표는 김성훈입니다.,업스테이지의 대표는 김성훈입니다.,1.000000e+00,1.000000,0.999998,1.000000,1.00
5,AI 가전의 3가지 핵심가치는?,"[사용자를 이해하고 돌보며 문제를 해결해주는 AI 홈 경험을 제공하겠다""고 강조했다...",AI 가전의 3가지 핵심 가치는 △누구나 쉽게 사용할 수 있고(Easy to Use...,- 누구나 쉽게 사용할 수 있음(Easy to Use)\n- 사용자를 돌봄(Care...,2.460137e-01,1.000000,0.724080,1.000000,1.00
6,지브리 풍의 그림을 그려주는 AI는 무엇입니까?,"[화제의 챗GPT-4o ‘지브리풍 그림’ 생성 기능, 지브리 저작권 침해 논란 입력...",지브리 풍의 그림을 그려주는 AI는 오픈AI의 ‘챗GPT-4o 이미지 생성’ 모델입니다.,챗GPT-4o 이미지 생성 모델이 지브리풍 그림을 그려주는 AI입니다.,8.913766e-02,1.000000,0.828385,1.000000,1.00
7,ChatGPT가 처한 저작권 위기는 무엇입니까?,[PICK 안내 언론사가 심층기획 기사로선정한 기사입니다. 언론사별 바로가기 닫기 ...,ChatGPT가 처한 저작권 위기는 다음과 같습니다.\n\n챗GPT의 이미지 생성 ...,- 발단: 챗GPT-4o의 이미지 생성 기능이 사용자 사진을 ‘지브리·디즈니·심슨’...,3.006541e-03,1.000000,0.811039,1.000000,0.59


In [ ]:
# dict로 변경하기(복잡..)
import ast

result_str = str(result)
result_dict = ast.literal_eval(result_str)
result_dict

{'bleu_score': 0.2173,
 'context_recall': 0.9031,
 'semantic_similarity': 0.8369,
 'faithfulness': 0.9817,
 'factual_correctness(mode=f1)': 0.7975}

# [실습] 다양한 파이프라인 파라미터 수정하기

현재 파이프라인에는 매우 다양한 조절 가능한 파라미터가 있습니다.   

각각의 파라미터를 수정하여, 성능 변화를 확인해 보세요.

Ex)
- Chunk의 크기/개수 늘리기   
- 모델 바꾸기   
- ...

In [ ]:
MODEL_NAME='gpt-4.1-mini'
TEMPERATURE=0
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 400
TOP_K = 5

# CSV 파일 경로 설정
csv_path = "experiments/experiment_log.csv"
os.makedirs("experiments", exist_ok=True)

def RAG_pipeline():

    dataset = []

    llm = ChatOpenAI(model=MODEL_NAME, temperature = TEMPERATURE, max_tokens = 4096)

    # 하이퍼파라미터별 고유 폴더명 생성
    DB_DIR = f"db_T{TEMPERATURE}_C{CHUNK_SIZE}_O{CHUNK_OVERLAP}_K{TOP_K}"
    try:
        os.listdir(DB_DIR)
        print('## 기존 DB 확인됨! 그대로 사용')
        db = Chroma(
            embedding_function=openai_embeddings,
            persist_directory=DB_DIR,
            collection_metadata={'hnsw:space': 'l2'},
        )
    except:
        print("## 기존 DB 없음, DB 새로 생성중...")

        preprocessed_docs = load_docs_from_jsonl("eval.jsonl")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        print('## DB 구성 중...')

        chunks = text_splitter.split_documents(preprocessed_docs)
        print(f'## Chunks 생성 완료... 총 {len(chunks)} 개 청크')
        db = Chroma(
            embedding_function=openai_embeddings,
            persist_directory=DB_DIR,
            collection_metadata={'hnsw:space': 'l2'},
        )
        for i in tqdm(range(0, len(chunks), 100)):
            db.add_documents(chunks[i:min(i+100, len(chunks))])
    print('## DB 구성 완료...')

    retriever = db.as_retriever(search_kwargs={'k':TOP_K})

    rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser())

    for query,reference in tqdm(zip(questions,ground_truths)):

        relevant_docs = [doc.page_content for doc in retriever.invoke(query)]
        response = rag_chain.invoke(query)
        dataset.append(
            {
                "user_input":query,
                "retrieved_contexts":relevant_docs,
                "response":response,
                "reference":reference
            }
        )

    print('## 답변 생성 완료...')
    evaluation_dataset = EvaluationDataset.from_list(dataset)

    result = evaluate(dataset=evaluation_dataset,
                    metrics=[BleuScore(), LLMContextRecall(), semantic_scorer, Faithfulness(), FactualCorrectness()],
                    llm=evaluator_llm,
                    embeddings = evaluator_embeddings
                    )

    result_str = str(result)
    result_dict = ast.literal_eval(result_str)
    return result_dict


실험 시작 시간과 정보를 저장할 수 있습니다.

In [ ]:
import csv
import datetime
import os

result = RAG_pipeline()

timestamp = datetime.datetime.now().isoformat()
row = {
    "timestamp": timestamp,
    "model_name": MODEL_NAME,
    "temperature": TEMPERATURE,
    "chunk_size": CHUNK_SIZE,
    "chunk_overlap": CHUNK_OVERLAP,
    "top_k": TOP_K,
    **result
}

# 파일이 없으면 헤더 추가
write_header = not os.path.exists(csv_path)

with open(csv_path, "a", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=row.keys())
    if write_header:
        writer.writeheader()
    writer.writerow(row)

print(f"[{timestamp}] {MODEL_NAME} | temp={TEMPERATURE}, chunk={CHUNK_SIZE}/{CHUNK_OVERLAP}, top_k={TOP_K} | results={result}")


## 기존 DB 없음, DB 새로 생성중...
## DB 구성 중...
## Chunks 생성 완료... 총 338 개 청크


100%|██████████| 4/4 [00:11<00:00,  2.91s/it]


## DB 구성 완료...


8it [00:41,  5.24s/it]


## 답변 생성 완료...


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

[2025-10-21T02:36:06.707540] gpt-4.1-mini | temp=0, chunk=2000/400, top_k=5 | results={'bleu_score': 0.0564, 'context_recall': 0.8376, 'semantic_similarity': 0.7795, 'faithfulness': 0.9889, 'factual_correctness(mode=f1)': 0.62}


다양한 기법을 위 코드에서 적용하여 성능을 높일 수 있습니다.